# API for the RSP from NOIRLab's Astro DataLab (Short Version)

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=190 style="padding: 10px">
<br>

**Contact authors:** Leanne Guy, Melissa Graham <br>
**Last verified:** Fri Dec 1 2023 <br>
**Recommended platform:** NOIRLab's Astro DataLab  (https://datalab.noirlab.edu/)<br>
**Kernel for recommended platform:** Python3 <br>
**Rubin data release:** Data Preview 0.2 (DP0.2) <br>

**This is a short, advanced version of the `api_from_noirlab.ipynb` notebook.**

**Description:**
This tutorial demonstrates how to access LSST-like data via the RSP from the NOIRLab Astro DataLab.
The scenario starts with data that would be in an LSST alert packet, then does a remote query
for additional data.

**Credit:** Sections 1 and 2 are based on the <a href="https://dp0-2.lsst.io/data-access-analysis-tools/api-intro.html">Introduction to the RSP API Aspect</a> webpage which had major contributions from Douglas Tucker.

**Requirements:** Accounts in the NOIRLab Astro DataLab and the Rubin Science Platform (RSP) at https://data.lsst.cloud/. Only individuals with <a href="https://docushare.lsst.org/docushare/dsweb/Get/RDO-013">Rubin data rights</a> may have an RSP account. See the <a href="https://dp0-2.lsst.io/dp0-delegate-resources/index.html#delegate-homepage-getting-started-checklist">getting started with DP0.2 checklist</a> for instructions about how to request an RSP account.

## 1. Set up RSP token in DataLab

Start a JupyterLab session at NOIRLab's Astro DataLab (https://datalab.noirlab.edu/).

**THE TOKEN IS A PASSWORD.**
Keep it secret. Keep it safe.

Follow steps 1 through 5 of <a href="https://nb.lsst.io/environment/tokens.html#using-a-token-outside-the-science-platform">these instructions to obtain a token for an RSP account</a>.
Be sure to select the box for "read:image".

Create a hidden file `~/.rsp-tap.token` containing only the token.
Use `chmod 600 .rsp-tap.token` to ensure user read/write permissions only.

## 2. Set up RSP TAP service

### 2.1. Import packages

Import <a href="https://pyvo.readthedocs.io/en/latest/">PyVO</a>,
<a href="https://docs.python.org/3/library/urllib.htmlhttps://docs.python.org/3/library/urllib.html">urllib</a>,
and a variety of other common packages.

In [ ]:
import pyvo
from pyvo.dal.adhoc import DatalinkResults
from pyvo.dal.adhoc import SodaQuery
import os, getpass
import pandas
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlretrieve
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS

### 2.2. Set up RSP TAP credentials

Define the filename of the token file created in Section 1.

In [ ]:
my_username = getpass.getuser()
token_filename = os.getenv('HOME')+'/.rsp-tap.token'
assert os.path.exists(token_filename)

Get the token from the token file.

In [ ]:
with open(token_filename, 'r') as f:
    token = f.readline()
assert token is not None

**Do not** `print(token)`.

Establish the RSP TAP service (`rsp_tap`) using `pyvo`. 

In [ ]:
cred = pyvo.auth.CredentialStore()
cred.set_password("x-oauth-basic", token)
credential = cred.get("ivo://ivoa.net/sso#BasicAA")
rsp_tap_url = 'https://data.lsst.cloud/api/tap'
rsp_tap = pyvo.dal.TAPService(rsp_tap_url, credential)
assert rsp_tap is not None
assert rsp_tap.baseurl == rsp_tap_url

### 2.3. Optional DP0.2 test query

In [ ]:
# query = "SELECT * FROM tap_schema.schemas"
# results = rsp_tap.run_sync(query).to_table()

In [ ]:
# results

In [ ]:
# del results

### 2.4. Optional DP0.3 test query

In [ ]:
# rsp_tap_url_sso = 'https://data.lsst.cloud/api/ssotap'
# rsp_tap_sso = pyvo.dal.TAPService(rsp_tap_url_sso, credential)
# assert rsp_tap_sso is not None
# assert rsp_tap_sso.baseurl == rsp_tap_url_sso

In [ ]:
# query_sso = "SELECT * FROM tap_schema.schemas"
# results_sso = rsp_tap_sso.run_sync(query_sso).to_table()

In [ ]:
# results_sso

In [ ]:
# del results_sso, rsp_tap_url_sso, rsp_tap_sso, query_sso

## 3. Obtain potential host galaxy information

Pretend the `diaObjectId`, and the `objectId` for the three nearest
extended objects from the latest data release, are known for five
transients of interest.

**In the future, this data would be in an LSST alert packet.**

### 3.1. Simulate nearby-object data from LSST alert packets

Create a pandas dataframe containing:

 * `diaObjectId` : identifier in the DP0.2 `DiaObject` table for the candidate supernova
 * `diaObject_coord` : coordinates [RA, Dec] in decimal degrees for the candidate supernova
 * `gals_objId` : identifier in the DP0.2 `Object` table for the three nearest galaxies
 * `gals_2Ddist` : the 2D sky distance in arcseconds between `DiaObject` and nearby galaxy's center

for five potential Type Ia supernovae in DP0.2.

In [ ]:
d = {'diaObjectId' : [1568026726510894110, 1569909090417642499, 1653700672547196623, 
                      1734140943235288573, 1825796232526695593],
     'diaObject_coord' : [[63.6025914, -38.634654],
                          [69.9257038, -38.1424959],
                          [70.8210894, -35.9915118],
                          [52.5432991, -34.9028848],
                          [71.7356252, -34.2191764]],
     'gals_objId' : [[1568026726510919266, 1568026726510919261, 1568026726510919497],
                     [1569425305301455007, 1569425305301455003, 1569425305301455014],
                     [1653700672547231391, 1653700672547231402, 1653700672547231397],
                     [1734140943235326493, 1734140943235293084, 1734140943235326492],
                     [1739084347513803559, 1739084347513803574, 1739084347513803571]],
     'gals_2Ddist' : [[3.15, 4.62, 5.08],
                      [0.02, 3.08, 3.98],
                      [2.13, 2.58, 4.34],
                      [4.7, 5.98, 6.11],
                      [0.03, 4.64, 5.93]]}
df = pandas.DataFrame(data=d)
del d

Option to display the dataframe.

In [ ]:
# df

### 3.2. Retrieve object data from the RSP's DP0.2 catalog

Choose to continue with the first `diaObject` in the dataframe.

In [ ]:
diao_index = 0

Create `list_objId`, a string containing a comma-separated list of the three `objectId` for
the three nearest galaxies to the selected `DiaObject`.

In [ ]:
temp = np.asarray(df['gals_objId'][diao_index], dtype='int')
list_objId = "(" + ','.join(['%20i' % num for num in temp]) + ")"
del temp
print(list_objId)

Create a query to retreive object astrometry, shape, size, and photometry measurements from the DP0.2 `Object` catalog.
See the <a href="https://dp0-2.lsst.io/data-products-dp0-2/index.html#dp0-2-data-products-definition-document-dpdd">DP0.2 DPDD</a> and <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 schema browser</a> for more information about the columns.

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, refExtendedness, "\
        "shape_xx, shape_xy, shape_yy, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS g_cModelMag, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS r_cModelMag, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS i_cModelMag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE objectId IN "+list_objId
del list_objId

Execute the query using the `rsp_tap` service, and store the results in `galaxies` as a table.

In [ ]:
galaxies = rsp_tap.search(query).to_table()

Option to view the remotely retrieved data table.

In [ ]:
# galaxies

### 3.3. Calculate additional galaxy properties

#### 3.3.1. Galaxy colors

Calculate the colors of the galaxies in $g-r$ and $r-i$ magnitude, and add them to the `galaxies` table.

In [ ]:
galaxies['gr_clr'] = galaxies['g_cModelMag'] - galaxies['r_cModelMag']
galaxies['ri_clr'] = galaxies['r_cModelMag'] - galaxies['i_cModelMag']

In [ ]:
# galaxies

#### 3.3.2. Separation in elliptical radii

**In the future, the LSST alert packet will contain separation distances in elliptical radii
that are based on the second moments of the galaxy's luminosity profile.**

Calculate them and add them to the `galaxies` table.

In [ ]:
galaxies['ell_rad'] = np.zeros(3, dtype='float')
galaxies['2Ddist'] = np.zeros(3, dtype='float')

snra = df['diaObject_coord'][diao_index][0]
sndec = df['diaObject_coord'][diao_index][1]
sncoord = SkyCoord(snra, sndec, unit='deg')

for i in range(3):
    objra = galaxies['coord_ra'][i]
    objdec = galaxies['coord_dec'][i]
    objcoord = SkyCoord(objra, objdec, unit='deg')
    del objra, objdec
    
    temp = objcoord.separation(sncoord)
    galaxies['2Ddist'][i] = temp.arcsec
    del temp
    
    temp = objcoord.spherical_offsets_to(sncoord)
    xr = 3600.0 * temp[0].deg
    yr = 3600.0 * temp[1].deg
    del temp, objcoord
    
    Ixx = galaxies['shape_xx'][i]
    Iyy = galaxies['shape_yy'][i]
    Ixy = galaxies['shape_xy'][i]
    Cxx = Iyy / ((Ixx * Iyy) - Ixy)
    Cyy = Ixx / ((Ixx * Iyy) - Ixy)
    Cxy = -2.0 * (Ixy) / ((Ixx * Iyy) - Ixy)
    galaxies['ell_rad'][i] = np.sqrt((Cxx * xr**2) + (Cyy * yr**2) + (Cxy * xr * yr))

    del Ixx, Iyy, Ixy, Cxx, Cyy, Cxy

del snra, sndec, sncoord

In [ ]:
galaxies

### 3.4. Interpret derived data for nearby galaxies

For the nearest galaxy by 2D sky separation, print the `objectId`, separations, and colors.

In [ ]:
mx = np.argmin(galaxies['2Ddist'])
print(galaxies['objectId'][mx], 
      galaxies['2Ddist'][mx], galaxies['ell_rad'][mx],
      galaxies['gr_clr'][mx], galaxies['ri_clr'][mx])
del mx

For the nearest galaxy by elliptical radii separation, print the `objectId`, separations, and colors.

In [ ]:
mx = np.argmin(galaxies['ell_rad'])
print(galaxies['objectId'][mx], 
      galaxies['2Ddist'][mx], galaxies['ell_rad'][mx],
      galaxies['gr_clr'][mx], galaxies['ri_clr'][mx])
del mx

**Summary:** The galaxy that is nearest by 2D sky separation (3.15") is not the best candidate host galaxy:
the best candidate is the one with a larger 2D sky separation (4.62") but a smaller offset in 
terms of elliptical radii ($R=1.02$), which takes into account the size of the galaxy.
Furthermore, the best candidate also has redder colors, ($g-r$ and $r-i>0$), which is more typical 
for the host galaxies of Type Ia supernovae.

In [ ]:
del diao_index, galaxies

## 4. Retrieve and display a light curve

**In the future, the LSST alert packet will contain the light curves.**

As a demo, retrieve the difference-image 5-sigma detections
for one of the `DiaObjects` and plot the light curve.

### 4.1. Set light curve plot parameters

These color and symbol combinations are colorblind-friendly.

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

### 4.2. Retrieve the light curve for one `DiaObject`

Descriptions and units for the columns of the `DiaSource` catalog are
available in the <a href="https://dm.lsst.org/sdm_schemas/browser/dp02.html">DP0.2 schema browser</a>.

In [ ]:
diao_index = 0
temp_string = str(df['diaObjectId'][diao_index])
diasources = rsp_tap.search("SELECT midPointTai, filterName, "
                            "scisql_nanojanskyToAbMag(psFlux) AS psAbMag "
                            "FROM dp02_dc2_catalogs.DiaSource  "
                            "WHERE diaObjectId = "+temp_string+" ").to_table()
del diao_index, temp_string

Option to display the data retrieved from the Rubin Science Platform.

In [ ]:
# diasources

### 4.3. Display the light curve

Plot the light curve with matplotlib. 

In [ ]:
fig = plt.figure(figsize=(6, 4))

for f, filt in enumerate(plot_filter_labels):
    fx = np.where(diasources['filterName'][:] == filt)[0]
    if len(fx) > 0:
        plt.plot(diasources['midPointTai'][fx]-60965, 
                 diasources['psAbMag'][fx],
                 plot_filter_symbols[filt], 
                 ms=10, mew=0, alpha=0.5,
                 color=plot_filter_colors[filt],
                 label=plot_filter_labels[f])
    del fx

plt.xlabel('Days Ago (MJD-60965)')
plt.ylabel('Apparent AB Magnitude')
plt.gca().invert_yaxis()
plt.title('5-Sigma Difference-Image Detections')
plt.legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
del diasources

## 5. Retrieve and display a large, deep cutout image

**In the future, the LSST alert packet will 
contain the difference-image and the reference-image stamps.**
However, they are relatively small cutouts: no smaller than 30 x 30 pixels
(6" x 6").

Obtain larger version to explore the transient's environment.

Select the first `DiaObject` on the list, and obtain its coordinates.

In [ ]:
diao_index = 0
snra = df['diaObject_coord'][diao_index][0]
sndec = df['diaObject_coord'][diao_index][1]
print(snra, sndec)

### 5.1. Query for the r-band deepCoadd image

Create the query for all deeply coadded image (`deepCoadd_calexp`) that overlaps the `DiaObject` coordinates, and then select the r-band.

It is recommended to always select all rows with `*` when querying the `ivoa.ObsCore`.

In [ ]:
query = '''SELECT * FROM ivoa.ObsCore 
WHERE dataproduct_type = 'image' 
AND obs_collection = 'LSST.DP02' 
AND dataproduct_subtype = 'lsst.deepCoadd_calexp' 
AND CONTAINS(POINT('ICRS', {}, {}), s_region) = 1
'''.format(snra, sndec)
print(query)

Query the `ivoa.ObsCore`, which is the DP0.2 images available via the Rubin Science Platform deployed in the Google Cloud.

Assert that the `results` table contains only six rows, one per filter.
Adding `AND lsst_band = 'r'` to the query above would result in just one row, for the r-band filter.

In [ ]:
results = rsp_tap.search(query)
assert len(results) == 6

Show the results in a table.

In [ ]:
results.to_table().show_in_notebook()

Store the results for r-band, which is the third row (index=2).

In [ ]:
results_r = results[2]

The `access_url` column contains the URL for the retrievable image, 
and the `access_format` specifies the format (content type) of the data product.

In [ ]:
print(results_r.getdataurl())
print(results_r.getdataformat())

The `access_format` indicates that the `access_url` is a <a href="https://www.ivoa.net/documents/DataLink/">DataLink</a> service.
DataLink is an IVOA data access protocol that provides a linking mechanism to metadata about a dataset, and the dataset itself.

### 5.2. Use DataLink to retrieve the image information

From `results_r`, get the data URL and print it.

In [ ]:
dl_url = results_r.getdataurl()
print('Datalink link service URL: ', dl_url)

Pass the Datalink ULR and the RSP credentials to `DatalinkResults`.

Executing the following cell gets the infomation stored at the datalink URL for this image:
a list matching records, each record containing a set of metadata describing the record.

Display the results as a table in the notebook, and see that there
are two rows: one contains the `access_url` for the exact storage
location for the full image data file,
and the other contains the `ID` for the cutout service (SODA) for this image.

In [ ]:
dl_results = DatalinkResults.from_result_url(dl_url, session=credential)
dl_results.to_table().show_in_notebook()

### 5.3. Download and display the full image

Download the full image, save it to file, and display it with matplotlib.

First, get the Google-signed URL for the image, and store it in `image_url`. 
Note that this is temporary and will expire.

In [ ]:
image_url = dl_results.getrecord(0).get('access_url')
print(image_url)

Retrieve the image save it to the current working directory 
(`getcwd`) with the filename "image.fits".

In [ ]:
image_file = os.path.join(os.getcwd(), 'image.fits')
urlretrieve(image_url, image_file)

Read the image header into `img_hdr`,
store the WCS in `img_wcs`,
and read the pixel data into `img_data`.

In [ ]:
hdulist = fits.open(image_file)
img_hdr = hdulist[1].header
img_wcs = WCS(img_hdr)
img_data = fits.getdata(image_file)

Option to view the contents of the image header.

In [ ]:
# img_hdr

Get the pixel coordinates of the supernova.

In [ ]:
sncoord = SkyCoord(snra, sndec, unit='deg')
snxy = img_wcs.world_to_pixel(sncoord)
print(snxy[0], snxy[1])

Display the image and mark the location of the supernova with a cyan cross.

> **Warning:** A pink deprecation warning for matplotlib might appear below. It is ok to ignore in the context of this tutorial.

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
plt.subplot(projection=img_wcs)
plt.imshow(img_data, cmap='gray', 
           vmin=0, vmax=0.5, norm='linear',
           origin='lower')
plt.plot(snxy[0], snxy[1], 'o', ms=14, color='none', mec='cyan')

Clean up; only `dl_results` is needed for the next section.

In [ ]:
del query, results, results_r, dl_url, image_url
del image_file, hdulist, img_hdr, img_wcs, img_data
del sncoord, snxy

### 5.3. Use SODA to make and retrieve a cutout image

Use the PyVO `SodaQuery` function to make a small cutout 
centered on the supernova, and *then* 
retrieve, save, and display the small cutout,
instead of retrieving the full image.

Get a link to the cutout service (`cutoutService`) for the r-band image
returned and stored in `dl_results`, above.

In [ ]:
cutoutService = dl_results.get_adhocservice_by_id("cutout-sync")
cutoutService.params

Create a SODA query (`sodaQuery`) service from the Datalink results
and the cutout service by passing the RSP credentials.

In [ ]:
sodaQuery = SodaQuery.from_resource(dl_results, cutoutService, session=credential)
assert sodaQuery is not None

Define the center and radius for the cutout.
Use the supernova's coordinates, and a radius
of about 30 arcsec (0.008 deg).

In [ ]:
sodaQuery.circle = (snra, sndec, 0.008)
print(sodaQuery.circle)
print(sodaQuery)

Define the location of the cutout image file.

In [ ]:
cutout_file = os.path.join(os.getcwd(), 'image-cutout.fits')

Execute the defined SODA query to make the cutout, 
read the results (the cutout image), and
save the cutout to the `cutout_file`.

In [ ]:
with open(cutout_file, 'bw') as f:
    f.write(sodaQuery.execute_stream().read())

Obtain the cutout image's header, WCS, and data.

Convert the supernova's coordinates to pixel coordinates for the cutout image.

In [ ]:
hdulist = fits.open(cutout_file)
coimg_hdr = hdulist[1].header
coimg_wcs = WCS(coimg_hdr)
coimg_data = fits.getdata(cutout_file)
sncoord = SkyCoord(snra, sndec, unit='deg')
snxy = coimg_wcs.world_to_pixel(sncoord)
print(snxy[0], snxy[1])

Display the cutout image.

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 5))
plt.subplot(projection=coimg_wcs)
plt.imshow(coimg_data, cmap='gray', 
           vmin=-0.1, vmax=0.5, norm='linear',
           origin='lower')
plt.plot(snxy[0], snxy[1], 'o', ms=14, color='none', mec='cyan')